In [0]:

.
PROJECT_ROOT = "s3://databricks-aryaman/xml-etl"

AUTOLOADER_SCHEMA = f"{PROJECT_ROOT}/_autoloader/schema/xml_raw/"
AUTOLOADER_CKPT   = f"{PROJECT_ROOT}/_autoloader/checkpoints/xml_raw/"

LANDING_XML       = f"{PROJECT_ROOT}/landing/xml/"
DELTA_BRONZE_PATH = f"{PROJECT_ROOT}/delta/bronze/"
DELTA_SILVER_PATH = f"{PROJECT_ROOT}/delta/silver/"
DELTA_GOLD_PATH   = f"{PROJECT_ROOT}/delta/gold/"

QUARANTINE        = f"{PROJECT_ROOT}/quarantine/"
LOGS              = f"{PROJECT_ROOT}/logs/"

In [0]:
spark.sql("SHOW CATALOGS").show()


+-------+
|catalog|
+-------+
|    aws|
|samples|
| system|
+-------+



In [0]:
%sql
SHOW SCHEMAS IN aws;


databaseName
default
information_schema


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS aws.bronze;
CREATE SCHEMA IF NOT EXISTS aws.silver;
CREATE SCHEMA IF NOT EXISTS aws.gold;

In [0]:
spark.readStream.format("cloudFiles")


In [0]:
%sql
SHOW STORAGE CREDENTIALS;


name,comment
aws,null


In [0]:
%sql
DESCRIBE STORAGE CREDENTIAL `aws`;

name,owner,created_at,created_by,credential,comment
aws,_workspace_admins_aws_7474645995236897,2026-01-15T02:00:22.667Z,tomeraryaman2004@gmail.com,AwsIamRole:arn:aws:iam::500325432803:role/DatabricksTrust,null


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS aws.landing;

In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS xml_etl_loc
URL 's3://databricks-aryaman/xml-etl'
WITH (STORAGE CREDENTIAL aws_xml_etl);

GRANT READ FILES, WRITE FILES
ON EXTERNAL LOCATION xml_etl_loc
TO `account users`;


In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS aws.landing.xml_etl_vol
LOCATION 's3://databricks-aryaman/xml-etl';


In [0]:
%sql
DESCRIBE EXTERNAL LOCATION xml_etl_loc;

name,url,credential_name,owner,created_by,created_at,comment
xml_etl_loc,s3://databricks-aryaman/xml-etl,aws,tomeraryaman2004@gmail.com,tomeraryaman2004@gmail.com,2026-01-16T17:27:42.23Z,null


In [0]:
%sql
ALTER EXTERNAL LOCATION xml_etl_loc
SET STORAGE CREDENTIAL aws_xml_etl;


In [0]:
%sql
DESCRIBE EXTERNAL LOCATION xml_etl_loc;

name,url,credential_name,owner,created_by,created_at,comment
xml_etl_loc,s3://databricks-aryaman/xml-etl,aws_xml_etl,tomeraryaman2004@gmail.com,tomeraryaman2004@gmail.com,2026-01-16T17:27:42.23Z,null


In [0]:
display(dbutils.fs.ls("/Volumes/aws/landing/xml_etl_vol/"))



path,name,size,modificationTime
dbfs:/Volumes/aws/landing/xml_etl_vol/_autoloader/,_autoloader/,0,1768590207784
dbfs:/Volumes/aws/landing/xml_etl_vol/delta/,delta/,0,1768590207784
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/,landing/,0,1768590207784
dbfs:/Volumes/aws/landing/xml_etl_vol/logs/,logs/,0,1768590207784
dbfs:/Volumes/aws/landing/xml_etl_vol/quarantine/,quarantine/,0,1768590207784


In [0]:
display(dbutils.fs.ls("/Volumes/aws/landing/xml_etl_vol/landing/xml/"))


path,name,size,modificationTime
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-08-29 17-21-39-308_stopped.xml,Workflow 2024-08-29 17-21-39-308_stopped.xml,7340,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-11 14-23-08-721_ReviewersStep.xml,Workflow 2024-09-11 14-23-08-721_ReviewersStep.xml,4608,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-18 19-16-59-848_Completed.xml,Workflow 2024-09-18 19-16-59-848_Completed.xml,7213,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-25 23-33-11-463_authorized.xml,Workflow 2024-09-25 23-33-11-463_authorized.xml,6340,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-27 14-15-04-910_authorized.xml,Workflow 2024-09-27 14-15-04-910_authorized.xml,10481,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-27 19-24-23-239_cancelled.xml,Workflow 2024-09-27 19-24-23-239_cancelled.xml,3521,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 12-06-16-969_ReviewersStep.xml,Workflow 2024-09-30 12-06-16-969_ReviewersStep.xml,4802,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 15-09-55-646_Completed.xml,Workflow 2024-09-30 15-09-55-646_Completed.xml,14713,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 16-04-13-538_approversStep.xml,Workflow 2024-09-30 16-04-13-538_approversStep.xml,3657,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 16-53-55-466_preProcessing.xml,Workflow 2024-09-30 16-53-55-466_preProcessing.xml,3555,1768495595000


In [0]:
dbutils.fs.mkdirs("/Volumes/aws/landing/xml_etl_vol/incoming")
dbutils.fs.mkdirs("/Volumes/aws/landing/xml_etl_vol/_checkpoints")
dbutils.fs.mkdirs("/Volumes/aws/landing/xml_etl_vol/_schemas")


True

In [0]:
display(dbutils.fs.ls("/Volumes/aws/landing/xml_etl_vol/"))


path,name,size,modificationTime
dbfs:/Volumes/aws/landing/xml_etl_vol/_autoloader/,_autoloader/,0,1768593865472
dbfs:/Volumes/aws/landing/xml_etl_vol/_checkpoints/,_checkpoints/,0,1768593865472
dbfs:/Volumes/aws/landing/xml_etl_vol/_schemas/,_schemas/,0,1768593865473
dbfs:/Volumes/aws/landing/xml_etl_vol/delta/,delta/,0,1768593865473
dbfs:/Volumes/aws/landing/xml_etl_vol/incoming/,incoming/,0,1768593865473
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/,landing/,0,1768593865473
dbfs:/Volumes/aws/landing/xml_etl_vol/logs/,logs/,0,1768593865473
dbfs:/Volumes/aws/landing/xml_etl_vol/quarantine/,quarantine/,0,1768593865473


In [0]:
%sql
DESCRIBE SCHEMA EXTENDED aws.bronze;


database_description_item,database_description_value
Catalog Name,aws
Namespace Name,bronze
Comment,
Location,
Owner,tomeraryaman2004@gmail.com
Properties,
Predictive Optimization,ENABLE (inherited from METASTORE metastore_aws_us_east_2)


In [0]:
%sql
GRANT USE CATALOG ON CATALOG aws TO `account users`;
GRANT USE SCHEMA ON SCHEMA aws.bronze TO `account users`;
GRANT CREATE TABLE ON SCHEMA aws.bronze TO `account users`;

In [0]:
%sql
GRANT USE CATALOG ON CATALOG aws TO `tomeraryaman2004@gmail.com`;
GRANT USE SCHEMA ON SCHEMA aws.bronze TO `tomeraryaman2004@gmail.com`;
GRANT CREATE TABLE, MODIFY, SELECT ON SCHEMA aws.bronze TO `tomeraryaman2004@gmail.com`;


In [0]:
%sql
DESCRIBE EXTERNAL LOCATION xml_etl_loc;

name,url,credential_name,owner,created_by,created_at,comment
xml_etl_loc,s3://databricks-aryaman/xml-etl,aws_xml_etl,tomeraryaman2004@gmail.com,tomeraryaman2004@gmail.com,2026-01-16T17:27:42.23Z,null


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS aws.bronze_uc;


In [0]:
%sql
GRANT USE CATALOG ON CATALOG aws TO `tomeraryaman2004@gmail.com`;
GRANT USE SCHEMA ON SCHEMA aws.bronze_uc TO `tomeraryaman2004@gmail.com`;
GRANT CREATE TABLE ON SCHEMA aws.bronze_uc TO `tomeraryaman2004@gmail.com`;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS aws.bronze.workflow_xml_raw (
  value STRING,
  _source_file STRING,
  ingested_at TIMESTAMP
)
USING DELTA;


In [0]:
%sql
DESCRIBE TABLE EXTENDED aws.bronze.workflow_xml_raw;


col_name,data_type,comment
value,string,null
_source_file,string,null
ingested_at,timestamp,null
,,
# Detailed Table Information,,
Catalog,aws,
Database,bronze,
Table,workflow_xml_raw,
Created Time,Sun Jan 18 18:11:57 UTC 2026,
Last Access,UNKNOWN,


In [0]:
%sql
DESCRIBE EXTERNAL LOCATION xml_etl_loc;


name,url,credential_name,owner,created_by,created_at,comment
xml_etl_loc,s3://databricks-aryaman/xml-etl,aws_xml_etl,tomeraryaman2004@gmail.com,tomeraryaman2004@gmail.com,2026-01-16T17:27:42.23Z,null


In [0]:
%sql
DESCRIBE DETAIL aws.bronze.workflow_xml_raw;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,aa69debe-bcc1-4146-9395-d4293a628989,aws.bronze.workflow_xml_raw,null,s3://databricks-aryaman/unity-catalog/7474645995236897/__unitystorage/catalogs/2a5346fd-cdb3-453b-bdec-f1346dc57114/tables/fb8d4405-4050-45f3-bc7b-7256698d211a,2026-01-18T18:11:52.999Z,2026-01-18T18:11:56Z,List(),List(),0,0,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true, delta.writePartitionColumnsToParquet -> true, delta.enableRowTracking -> true, delta.rowTracking.materializedRowCommitVersionColumnName -> _row-commit-version-col-2b55539d-86ae-41fa-a860-543378a88133, delta.rowTracking.materializedRowIdColumnName -> _row-id-col-29c2bc3a-9d9c-4d3e-a6c1-7ed593f836fb)",3,7,"List(appendOnly, deletionVectors, domainMetadata, invariants, rowTracking)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
spark.sql("INSERT INTO aws.bronze.workflow_xml_raw VALUES ('test', 'manual', current_timestamp())")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
from pyspark.sql.functions import input_file_name, current_timestamp, col

source_path = "/Volumes/aws/landing/xml_etl_vol/landing/xml/"
schema_loc   = "/Volumes/aws/landing/xml_etl_vol/_autoloader/schema_workflow_xml_raw/"
checkpoint   = "/Volumes/aws/landing/xml_etl_vol/_autoloader/checkpoints/workflow_xml_raw/"

df = (
  spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "binaryFile")
    .option("cloudFiles.schemaLocation", schema_loc)
    .load(source_path)
    .select(
        col("content").cast("string").alias("value"),
        input_file_name().alias("_source_file"),
        current_timestamp().alias("ingested_at")
    )
)

(df.writeStream
  .format("delta")
  .option("checkpointLocation", checkpoint)
  .trigger(availableNow=True)
  .toTable("aws.bronze.workflow_xml_raw")
)


In [0]:
display(dbutils.fs.ls("/Volumes/aws/landing/xml_etl_vol/landing/xml/"))


path,name,size,modificationTime
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-08-29 17-21-39-308_stopped.xml,Workflow 2024-08-29 17-21-39-308_stopped.xml,7340,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-11 14-23-08-721_ReviewersStep.xml,Workflow 2024-09-11 14-23-08-721_ReviewersStep.xml,4608,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-18 19-16-59-848_Completed.xml,Workflow 2024-09-18 19-16-59-848_Completed.xml,7213,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-25 23-33-11-463_authorized.xml,Workflow 2024-09-25 23-33-11-463_authorized.xml,6340,1768495594000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-27 14-15-04-910_authorized.xml,Workflow 2024-09-27 14-15-04-910_authorized.xml,10481,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-27 19-24-23-239_cancelled.xml,Workflow 2024-09-27 19-24-23-239_cancelled.xml,3521,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 12-06-16-969_ReviewersStep.xml,Workflow 2024-09-30 12-06-16-969_ReviewersStep.xml,4802,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 15-09-55-646_Completed.xml,Workflow 2024-09-30 15-09-55-646_Completed.xml,14713,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 16-04-13-538_approversStep.xml,Workflow 2024-09-30 16-04-13-538_approversStep.xml,3657,1768495595000
dbfs:/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow 2024-09-30 16-53-55-466_preProcessing.xml,Workflow 2024-09-30 16-53-55-466_preProcessing.xml,3555,1768495595000


In [0]:
%sql
SELECT COUNT(*) AS row_count FROM aws.bronze.workflow_xml_raw;


row_count
11


In [0]:
%sql
SELECT _source_file, ingested_at
FROM aws.bronze.workflow_xml_raw
ORDER BY ingested_at DESC
LIMIT 20;


_source_file,ingested_at
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-08-29%2017-21-39-308_stopped.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-25%2023-33-11-463_authorized.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2015-09-55-646_Completed.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-18%2019-16-59-848_Completed.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2012-06-16-969_ReviewersStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-27%2014-15-04-910_authorized.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2016-53-55-466_preProcessing.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2016-04-13-538_approversStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-11%2014-23-08-721_ReviewersStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-27%2019-24-23-239_cancelled.xml,2026-01-18T18:44:07.546Z


In [0]:
%sql
CREATE OR REPLACE VIEW aws.bronze.workflow_xml_raw_dedup AS
SELECT *
FROM (
  SELECT *,
         row_number() OVER (PARTITION BY _source_file ORDER BY ingested_at DESC) AS rn
  FROM aws.bronze.workflow_xml_raw
)
WHERE rn = 1;


In [0]:
%sql
SELECT _source_file, ingested_at
FROM aws.bronze.workflow_xml_raw_dedup
ORDER BY ingested_at DESC
LIMIT 20;


_source_file,ingested_at
manual,2026-01-18T20:47:18.997647Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-27%2014-15-04-910_authorized.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2012-06-16-969_ReviewersStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-18%2019-16-59-848_Completed.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-27%2019-24-23-239_cancelled.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-11%2014-23-08-721_ReviewersStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2016-04-13-538_approversStep.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-30%2015-09-55-646_Completed.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-08-29%2017-21-39-308_stopped.xml,2026-01-18T18:44:07.546Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-09-25%2023-33-11-463_authorized.xml,2026-01-18T18:44:07.546Z


In [0]:
%sql
SELECT
  _source_file,
  left(value, 40) AS value_prefix,
  length(value) AS value_len,
  ingested_at
FROM aws.bronze.workflow_xml_raw
WHERE value IS NULL
   OR trim(value) = ''
   OR lower(trim(value)) = 'manual'
   OR ltrim(value) NOT LIKE '<?xml%'
ORDER BY ingested_at DESC;




_source_file,value_prefix,value_len,ingested_at
manual,test,4,2026-01-18T20:47:18.997647Z
/Volumes/aws/landing/xml_etl_vol/landing/xml/Workflow%202024-08-29%2017-21-39-308_stopped.xml,"﻿<?xml version=""1.0"" encoding=""utf-8""?>",7338,2026-01-18T18:44:07.546Z


In [0]:
%sql
SELECT *
FROM aws.bronze.workflow_xml_raw
WHERE _source_file = 'manual';


value,_source_file,ingested_at
test,manual,2026-01-18T20:47:18.997647Z


In [0]:
%sql
DELETE FROM aws.bronze.workflow_xml_raw
WHERE _source_file = 'manual';


num_affected_rows
1


In [0]:
%sql
SELECT COUNT(*) 
FROM aws.bronze.workflow_xml_raw
WHERE _source_file = 'manual';


count(1)
0


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS aws.silver;
